<a href="https://colab.research.google.com/github/MoltenMuffins/IndoorAudioClassifier/blob/master/AudioAsImage_TransferLearningIncep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Shallow layers trained on self created database of sounds from freesound

Classes: Speech, Music, Water, Door Sounds, Car horn, Glass Breaking

## 0. Boilerplate Code

In [0]:
#test for gpu
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
#imports here
import os
import librosa
from tensorflow.keras.models import Model,load_model,Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K

import tensorflow as tf

tf.VERSION

'1.12.0'

## 1. Download Dataset

We use a small labeled dataset generated via the freesound api and passed through an audio tokeniser to shorten exceedingly long sound files.

It has the following file structure: `Dataset/Train/{CLASS_LABEL}/{FILENAME}.jpg`

In [0]:
#Download Dataset from dropbox
!wget -qq https://www.dropbox.com/s/rv7xzjyvae0nabt/Data.zip

In [0]:
#Unzip and delete zip file
!unzip -qq Data.zip
!rm Data.zip
!ls

Data  __MACOSX	sample_data


## 2. Prepare Dataset for Model

We use glob to get lists of the files in the directories and then convert them into dataframes and add in class numbers.

We also then split them up so we have 10% for a testing set and the rest for training.

Finally we randomly shuffle them up

In [0]:
path = "./Data/"
train_data_dir = "./Data/Train/"

In [0]:
import pandas as pd
import glob

filenames_n0 = glob.glob('./Data/Train/car horn/*.png')
filenames_n1 = glob.glob('./Data/Train/door/*.png')
filenames_n2 = glob.glob('./Data/Train/glass break/*.png')
filenames_n3 = glob.glob('./Data/Train/music/*.png')
filenames_n4 = glob.glob('./Data/Train/speech/*.png')
filenames_n5 = glob.glob('./Data/Train/water/*.png')


names = ['car horn', 'door', 'glass break', 'music', 'speech', 'water']

len(filenames_n0)

53

In [0]:
# Make a dataframe based on the filenames
df = pd.DataFrame(filenames_n0, columns = ["filename"])
df2 = pd.DataFrame(filenames_n1, columns = ["filename"])
df3 = pd.DataFrame(filenames_n2, columns = ["filename"])
df4 = pd.DataFrame(filenames_n3, columns = ["filename"])
df5 = pd.DataFrame(filenames_n4, columns = ["filename"])
df6 = pd.DataFrame(filenames_n5, columns = ["filename"])


# Add Class columns 
df['class'] = pd.Series([0 for x in range(len(df.index))], index=df.index)
df2['class'] = pd.Series([1 for x in range(len(df2.index))], index=df2.index)
df3['class'] = pd.Series([2 for x in range(len(df3.index))], index=df3.index)
df4['class'] = pd.Series([3 for x in range(len(df4.index))], index=df4.index)
df5['class'] = pd.Series([4 for x in range(len(df5.index))], index=df5.index)
df6['class'] = pd.Series([5 for x in range(len(df6.index))], index=df6.index)


# Split into train and validation sets
train_set_percentage = .9

#1
train_df = df[:int(len(df)*train_set_percentage)]
val_df = df[int(len(df)*train_set_percentage):]

#2
train_df2 = df2[:int(len(df2)*train_set_percentage)]
val_df2 = df2[int(len(df2)*train_set_percentage):]

#3
train_df3 = df3[:int(len(df3)*train_set_percentage)]
val_df3 = df3[int(len(df3)*train_set_percentage):]

#4
train_df4 = df4[:int(len(df4)*train_set_percentage)]
val_df4 = df4[int(len(df4)*train_set_percentage):]

#5
train_df5 = df5[:int(len(df5)*train_set_percentage)]
val_df5 = df5[int(len(df5)*train_set_percentage):]

#6
train_df6 = df6[:int(len(df6)*train_set_percentage)]
val_df6 = df6[int(len(df6)*train_set_percentage):]

df_new_train = pd.concat([train_df, train_df2, train_df3, train_df4, train_df5, train_df6])
df_new_val = pd.concat([val_df, val_df2, val_df3, val_df4, val_df5, val_df6])

In [0]:
# shuffle dataframes
df = df_new_train.sample(frac=1).reset_index(drop=True)
df_val = df_new_val.sample(frac=1).reset_index(drop=True)

print('number of train files:', len(df))
print('number of val files:', len(df_val))
df.head(10)

number of train files: 911
number of val files: 104


,filename,class
0,"./Data/Train/glass break/Glass Smash, Bottle, ...",2
1,./Data/Train/door/FrontDoor16.png,1
2,./Data/Train/glass break/breaking glass.png,2
3,"./Data/Train/speech/""Que importa"".png",4
4,./Data/Train/car horn/Van horn on rainy day_cl...,0
5,./Data/Train/speech/speech-female_hpsModelTran...,4
6,./Data/Train/door/Door - Full 3_clip0.png,1
7,./Data/Train/door/12_fast_knocks_on_computer_r...,1
8,./Data/Train/music/shaker ending_clip0.png,3
9,./Data/Train/speech/Ipa_MeowVoice_18_F4.png,4


In [0]:
# convert the dataframe into 2 lists to use for filename and labels
train_filenames_list = df["filename"].tolist()
train_labels_list = df["class"].astype('int32').tolist()

# convert the dataframe into 2 lists to use for filename and labels
val_filenames_list = df_val["filename"].tolist()
val_labels_list = df_val["class"].astype('int32').tolist()

#number of classes
num_classes = 6

df.shape

(911, 2)

In [0]:
train_filenames_list[:5]

['./Data/Train/glass break/Glass Smash, Bottle, E.png',
 './Data/Train/door/FrontDoor16.png',
 './Data/Train/glass break/breaking glass.png',
 './Data/Train/speech/"Que importa".png',
 './Data/Train/car horn/Van horn on rainy day_clip0.png']

## 3. Inceptionv3

### 3.1 Create Data Pipeline

In [0]:
# Reads an image from a file, decodes it into a tensor, and resizes it
# to a fixed shape.
img_rows, img_cols = 299,299

def _parse_function(filename, label):
  image_string = tf.read_file(filename)
  #Channels specified to be 3 to ensure images output as [299,299,3] at the end of this
  image_decoded = tf.image.decode_jpeg(image_string, channels=3)
  image_resized = tf.image.resize_images(image_decoded, [img_rows, img_cols])
  label = tf.one_hot(label, num_classes)
  return image_resized, label

In [0]:
# Create vector of filenames from list
filenames = tf.constant(train_filenames_list)

# Create vector of labels
labels = tf.constant(train_labels_list)

# Same as above but for validation set
val_filenames = tf.constant(val_filenames_list)
val_labels = tf.constant(val_labels_list)

### 3.2 Assemble Data Pipeline for InceptionV3

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
train_dataset = train_dataset.map(_parse_function)
train_dataset = train_dataset.repeat()
train_dataset = train_dataset.batch(32)

valid_dataset = tf.data.Dataset.from_tensor_slices((val_filenames, val_labels))
valid_dataset = valid_dataset.map(_parse_function)
valid_dataset = valid_dataset.repeat()
valid_dataset = valid_dataset.batch(32)

In [0]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
base_model.summary()

87916544/87910968 [==============================] - 5s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
____________________________________________

In [0]:
# add a global spatial average pooling layer
x = base_model.output

x = Flatten()(x)

# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)

# and a logits layer -- let's say we have 6 classes
predictions = Dense(6, activation='softmax')(x)

In [0]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [0]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional VGG16 layers
for layer in base_model.layers:
    layer.trainable = False
    
#for layer in model.layers:
#    print(layer.name)
#    print(layer.trainable)

In [0]:
#optional
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d_1 (

In [0]:
opt = tf.train.AdamOptimizer(learning_rate = 0.001)

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])

### 3.3 Transfer Learning on InceptionV3

In [0]:
train_steps = 270
val_steps = 90
epochs = 10

In [0]:
#sanity check
print(train_dataset)
print(valid_dataset)

<BatchDataset shapes: ((?, 299, 299, 3), (?, 6)), types: (tf.float32, tf.float32)>
<BatchDataset shapes: ((?, 299, 299, 3), (?, 6)), types: (tf.float32, tf.float32)>


In [0]:
# # # Train the model with validation 
history = model.fit( train_dataset, steps_per_epoch = train_steps,
                   epochs = epochs,
                   validation_data = valid_dataset,
                   validation_steps = val_steps)

Epoch 1/10
270/270 [==============================] - 145s 538ms/step - loss: 12.6969 - acc: 0.2094 - val_loss: 12.8441 - val_acc: 0.2031
Epoch 2/10
270/270 [==============================] - 138s 511ms/step - loss: 12.7695 - acc: 0.2078 - val_loss: 12.8777 - val_acc: 0.2010
Epoch 3/10
270/270 [==============================] - 137s 509ms/step - loss: 12.7972 - acc: 0.2059 - val_loss: 12.8665 - val_acc: 0.2017
Epoch 4/10
270/270 [==============================] - 137s 506ms/step - loss: 12.7602 - acc: 0.2083 - val_loss: 12.8497 - val_acc: 0.2028
Epoch 5/10
270/270 [==============================] - 137s 508ms/step - loss: 12.7825 - acc: 0.2069 - val_loss: 12.8721 - val_acc: 0.2014
Epoch 6/10
270/270 [==============================] - 139s 514ms/step - loss: 12.7714 - acc: 0.2076 - val_loss: 12.8721 - val_acc: 0.2014
Epoch 7/10
270/270 [==============================] - 139s 515ms/step - loss: 12.7714 - acc: 0.2076 - val_loss: 12.8497 - val_acc: 0.2028
Epoch 8/10
270/270 [==============

In [0]:
#epochs = 3

# # # Train the model with validation 
#history = model.fit( train_dataset, steps_per_epoch = train_steps,
                   #epochs = epochs,
                   #validation_data = valid_dataset,
                   v#alidation_steps = val_steps)

NameError: ignored

In [0]:
#metrics = model.evaluate(valid_dataset,
                   #steps = val_steps)
#print("model accuracy:",metrics[1])